# ソーシャルメディアテキストの分析

## 準備

In [ ]:
%%bash
apt-get install mecab libmecab-dev mecab-ipadic-utf8
pip install mecab-python3 ipadic nlplot japanize-matplotlib ginza ja-ginza
ln -s /etc/mecabrc /usr/local/etc/mecabrc

In [ ]:
import pandas as pd
import gensim
import re
import nlplot
import MeCab
mecab=MeCab.Tagger()
import spacy
nlp=spacy.load('ja_ginza',disable=['parser','ner']) # 形態素解析だけする設定

from collections import defaultdict

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import matplotlib
import plotly
from plotly.subplots import make_subplots
from plotly.offline import iplot
%matplotlib inline

In [ ]:
# Ginzaで内容語の抽出
def text2bow_ginza(sentence):
  words=[]
  doc=nlp(sentence)
  for token in doc:
    if token.pos_ in ['NOUN','VERB','ADJ','ADV','PROPN','PRON']:
      words.append(token.lemma_)
  return words

In [ ]:
# MeCabで内容語の抽出
def mecabparse(sentence):
  mecab_result=mecab.parse(sentence) # 形態素解析の実行
  mecab_result=mecab_result.rstrip() # 最後の改行の削除
  out=re.split('\n',mecab_result) # 改行で分割して1形態素毎のリストにする
  return(out)

def text2bow_mecab(sentence):
  words=[]
  morphs=mecabparse(sentence)
  for i in range(len(morphs)):
    if morphs[i]!='EOS':
      line=re.split('\t',morphs[i])
      features=re.split(',',line[1])
      if re.match('名詞|動詞|形容詞|副詞',features[0]) and not re.match('非自立|代名詞|接尾',features[1]):
        lemma=features[6]
        if lemma=='*':
          lemma=line[0]
        words.append(lemma)
  return words

# Tweetの分析

## データの読み込み
教材に置いてあるstream.20160123_\{00-16|17-23\}.tar.bz2をアップロードする．

このデータは前処理済みで，
```
アカウント名 \t tweet \t タイムスタンプ
```
となっている．

(各時間ごとにファイルを分割しているので時間単位でデータをまとめる場合はタイムスタンプは見なくても良い)


In [ ]:
%%bash
tar jxvf stream.20160123_00-16.tar.bz2
tar jxvf stream.20160123_17-23.tar.bz2

### MeCabとGinzaの比較
形態素解析だけに限って，MeCabとGinzaの実行速度を見てみる．


In [ ]:
import time

In [ ]:
df=pd.read_csv('/content/stream.20160123/stream.2016012300.tsv',names=('account','tweet','created_at'),sep='\t')
start=time.time()
df['tweet'].head(100).apply(text2bow_mecab)
end=time.time()
print('MeCab:',end-start)
start=time.time()
df['tweet'].head(100).apply(text2bow_ginza)
end=time.time()
print('Ginza:',end-start)

今回はMeCabを使う．

In [ ]:
data=defaultdict(list) # 時間をkey，各ファイルの内容のDataFrameをvalueとしたdict形式にする．
for i in range(0,24):
  filename='/content/stream.20160123/stream.20160123'+'%02d'%i+'.tsv'
  df=pd.read_csv(filename,names=('account','tweet','created_at'),sep='\t')
  df['words']=df['tweet'].apply(text2bow_mecab)
  data[i]=df

各時間ごとのtweetの数を見る．

In [ ]:
for i in range(0,24):
  print(i,len(data[i]),sep='\t')

## 各時間帯での頻出語を見る

……前に，まず1時間分だけ見てみる．

In [ ]:
npt=nlplot.NLPlot(data[0],target_col='words')
wc=npt.wordcloud(
    max_words=100,
    max_font_size=100,
    colormap='tab20_r'
)
plt.figure(figsize=(8, 6))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

### データの前処理
URLやRTなどが頻度上位に現れるのはあまり嬉しくない．

そもそもtweetがどういうテキストでいらなさそうなものが他にあるか見てみる．

In [ ]:
data[0].head(50)

どんな前処理をすれば良いかは目的によるが，今回は以下の方針で行う．
- URLは削除
- RTは"RT"だけ削除
- "@アカウント名"を削除

あらためてデータの読み込み．

In [ ]:
data=defaultdict(list) # 時間をkey，各ファイルの内容のDataFrameをvalueとしたdict形式にする．
for i in range(0,24):
  filename='/content/stream.20160123/stream.20160123'+'%02d'%i+'.tsv'
  account=[]
  tweet=[]
  timestamp=[]
  with open(filename,'r') as fh:
    for ln in fh:
      ln=ln.rstrip()
      line=re.split('\t',ln)
      line[1]=re.sub('https?://[\w/:%#\$&\?\(\)~\.=\+\-]+','',line[1])
      line[1]=re.sub('^RT ','',line[1])
      line[1]=re.sub('\@.+?\s','',line[1])
      if line[1]!='':
        account.append(line[0])
        tweet.append(line[1])
        timestamp.append(line[2])
  df=pd.DataFrame({'account':account,'tweet':tweet,'created_at':timestamp})
  df['words']=df['tweet'].apply(text2bow_mecab)
  data[i]=df

もう一度ワードクラウドを見てみる．

In [ ]:
npt=nlplot.NLPlot(data[0],target_col='words')
wc=npt.wordcloud(
    max_words=100,
    max_font_size=100,
    colormap='tab20_r'
)
plt.figure(figsize=(8, 6))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

24時間分見てみる．

In [ ]:
for i in range(0,24):
  print(i)
  npt=nlplot.NLPlot(data[i],target_col='words')
  wc=npt.wordcloud(
      max_words=100,
      max_font_size=100,
      colormap='tab20_r'
  )
  plt.figure(figsize=(8, 6))
  plt.imshow(wc, interpolation="bilinear")
  plt.axis("off")
  plt.show()

### tfidfで見てみる
各時間ごとの特徴語を調べるためにtfidfを見てみる．

In [ ]:
# tfとdfの計算
tf=defaultdict(lambda:defaultdict(int)) # 各時間あたりの単語の出現頻度
df=defaultdict(int) # 単語の文書頻度
n_words=defaultdict(int) # 各時間の単語数
for i in range(0,24):
  tmp=set() # 各時間あたりの単語の出現だけを見るのでlistでなくsetを使う
  for words in data[i]['words']:
    for w in words:
      tf[i][w]+=1
      n_words[i]+=1
      tmp.add(w)
  for t in tmp:
    df[t]+=1

In [ ]:
import math
tfidf=defaultdict(float)
freq=defaultdict(int)
with open('/content/natsume/吾輩は猫である.sentences','r') as fh:
  n_w=defaultdict(float) # テキスト中の単語の頻度
  n_d=0 # テキスト中の単語数
  for ln in fh:
    ln=ln.rstrip()
    for lemma in getwords(ln):
      n_w[lemma]+=1
      freq[lemma]+=1
      n_d+=1
  for w in n_w:
    # 頻度が1の単語は除外
    if n_w[w]==1:
      continue
    tf=n_w[w]/n_d
    idf=math.log(textnum/df[w])+1
    tfidf[w]=tf*idf


In [ ]:
import math
tfidf=defaultdict(lambda:defaultdict(float))

for i in range(0,24):
  for w in tf[i]:
    # 全てのテキストに現れる語は0にする
    tfidf[i][w]=(tf[i][w]/n_words[i])*(math.log(24/df[w]))

In [ ]:
# 上位20件ずつ表示
for i in range(0,24):
  print('====',i,'====')
  srted=sorted(tfidf[i].keys(),key=lambda x:tfidf[i][x],reverse=True)
  for j in range(20):
    print(srted[j],tfidf[i][srted[j]],sep='\t')


## 感情分析
評判分析のpos/negではなく，いわゆる感情．

pymlaskはML-Askのpython実装で，中村の10種類の感情分類に従った辞書ベースの感情推定手法．

各時間あたりの感情成文の分布を見てみる

In [ ]:
! pip install pymlask

pymlaskの簡単な使い方説明．

In [ ]:
from mlask import MLAsk
emotion_analyzer = MLAsk()
emolabel={'aware':'哀','iya':'厭','yorokobi':'喜','suki':'好','yasu':'安','takaburi':'昂','odoroki':'驚','ikari':'怒','kowa':'怖','haji':'恥'}

In [ ]:
emo=emotion_analyzer.analyze('京極の小説は大好きだが，本人はちょっと好きではない……')
cnt=defaultdict(int)
if emo['emotion'] is not None:
  for k,v in emo['emotion'].items():
    cnt[k]+=1
  for k in cnt:
    print(emolabel[k],cnt[k],sep='\t')

Tweetの感情分析．

In [ ]:
emodist=defaultdict(lambda:defaultdict(int))
for i in range(0,24):
  for t in data[i]['tweet']:
    emo=emotion_analyzer.analyze(t)
    if emo['emotion'] is not None:
      for k,v in emo['emotion'].items():
        emodist[i][k]+=1



In [ ]:
for i in range(0,24):
  print('====',i,'====')
  for k in emodist[i]:
    print(emolabel[k],emodist[i][k])


感情の割合の遷移を折れ線グラフで表示してみる．

In [ ]:
emotrans=defaultdict(list) # 各感情の時間ごとの割合を格納
for i in range(0,24):
  total=0
  for k in emodist[i]:
    total+=emodist[i][k] # 割合を計算するので総カウント数を求める
  for k in emolabel:
    if k in emodist[i]:
      emotrans[k].append(emodist[i][k]/total)
    else:
      emotrans[k].append(0.0)

In [ ]:
for k in emolabel:
  plt.plot(range(0,24),emotrans[k],marker='.',label=k)
plt.legend(loc = 'upper right')
plt.show()
